# In this draft script I will try to build a optimization workers with homogeneous interfaces
___

### It has built on top of [mlrose](https://github.com/gkhayes/mlrose) package



In [115]:
import mlrose
import numpy as np
import logging
import tsplib95
# import pandas as pd
RANDOM_SEED = 1

np.random.seed(seed=RANDOM_SEED)

### Function for generating the problem.
##### NOTE: The function should return the instance of initialized mlrose problem.
This part of notebook will be enhanced with adding other problem instances

In [117]:
def GenerteNQueensProblem(n: int = 8):
    problem = mlrose.DiscreteOpt(length=n, fitness_fn=mlrose.Queens(), maximize=False, max_val=n)
    return problem

def GenerateTSPProblem(instance_name: str="a280.tsp"):
    tsplib_problem = tsplib95.load_problem("TSPProblems/" + instance_name if not '/' == instance_name[0] else instance_name)
    cities_coordinates = list(tsplib_problem.node_coords.values())
    
    problem = mlrose.TSPOpt(length=tsplib_problem.dimension, coords=cities_coordinates, maximize=False)
    return problem
    

#### Simulated annealing

In [118]:
def RunSASolver(problem, init_state, budget, mh_parameters):
    assert "schedule" in mh_parameters.keys()
    assert "max_attempts" in mh_parameters.keys()
    
    best_state, best_fitness = mlrose.simulated_annealing(problem,
                                                          max_iters=budget,
                                                          init_state=init_state,
                                                          random_state=RANDOM_SEED,
                                                          **mh_parameters)
    return best_state, best_fitness

#### Stohastic hill climbing

In [119]:
def RunSCH(problem, init_state, budget, mh_parameters):
    results = mlrose.random_hill_climb(problem,
                                       init_state=init_state,
                                       restarts=budget,
                                       random_state=RANDOM_SEED,
                                       **mh_parameters)
    return results

#### Genetic algorithm

In [120]:
def RunGA(problem, init_state, budget, mh_parameters):
    best_state, best_fitness = mlrose.genetic_alg(problem,
                                                  max_iters=budget,
                                                  random_state=RANDOM_SEED,
                                                  **mh_parameters)
    return best_state, best_fitness

#### Running the algorithms
Just for testing purposes.

In [123]:
sa_mh_parameters = {
    'schedule': mlrose.GeomDecay(),
    'max_attempts': 1000,
}
sch_mh_parameters = {
    'max_attempts': 100
}
gen_mh_parameters = {'mutation_prob': 0.3, 'max_attempts': 30}

n_queen_problem = GenerteNQueensProblem(19)
tsp_problem = GenerateTSPProblem('/media/sem/B54BE5B22C0D3FA8/TUD/Master/metaheuristics_library/TSPProblems/kroA100.tsp')


budget = 30000
iterations = 1

for problem in (n_queen_problem, tsp_problem):
    for mh, params in zip((
         RunSASolver, 
         RunSCH, 
        RunGA,
    ), (
         sa_mh_parameters, 
         sch_mh_parameters, 
        gen_mh_parameters,
    )):
        init_state = np.array(list(range(19 if type(problem) == mlrose.DiscreteOpt else 280)))
        for iteration in range(iterations):
            solution, solution_fitness = mh(problem, init_state, budget=budget // iterations, mh_parameters=params)
            print("%s:%s, solution: %s, fitness: %s." % (mh.__name__, iteration, solution, solution_fitness))
            init_state = solution
        problem.reset()
        print("problem reset")


RunGA:0, solution: [ 8 11 15  7 11  4 15  0 10 14  2  2 16 13 17  5  3  1 17], fitness: 8.0.
problem reset
RunGA:0, solution: [75 28 34 59 91 87 21  7 52  4 29 36 16 71 55 26 46 77 79 48 89 56 67 24
 92 95 39 53 27 13 49 20 74 62 45 42 68 10 15 88 69 37 78 12 51 72 47 99
 11 22 94 70 61 23 43 63 85 35 18 93  0 96 81 32 50 58 41 90 98 17 38  6
 73  9 30 60 66 19 86  2 44 83  5  8 84 57 76 14 31 40  1 82 25 65 97  3
 80 54 33 64], fitness: 138584.63306358282.
problem reset


In [100]:
optimal = [0, 1, 241, 242, 243, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 245, 244, 246, 249, 250, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 206, 205, 204, 203, 202, 201, 200, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 175, 179, 178, 149, 177, 176, 150, 151, 155, 152, 154, 153, 128, 129, 130, 19, 20, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 156, 157, 158, 159, 174, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 171, 170, 172, 173, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 108, 107, 109, 110, 111, 87, 86, 112, 113, 114, 116, 115, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 58, 62, 61, 117, 60, 59, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 21, 24, 22, 23, 13, 14, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 276, 275, 274, 273, 272, 271, 270, 15, 16, 17, 18, 131, 132, 133, 269, 268, 134, 135, 267, 266, 136, 137, 138, 148, 147, 146, 145, 144, 198, 199, 143, 142, 141, 140, 139, 265, 264, 263, 262, 261, 260, 259, 258, 257, 256, 253, 252, 207, 208, 251, 254, 255, 248, 247, 277, 278, 2, 279]
print(solution_result)
print(problem.fitness_fn.evaluate(optimal))

30848.87087287119
2586.7696475631606
